In [ ]:
# https://adventofcode.com/2018/day/3

struct Claim
    id::String
    left::Int
    top::Int
    width::Int
    height::Int
end

" parse a claim string into a struct "
function Claim(formatted::String)
    cap = match(r"#(\d+) @ (\d+),(\d+): (\d+)x(\d+)", formatted).captures
    Claim(cap[1], [parse(Int, c) for c in cap[2:end]]...)
end

" test a claim rectangle for empty "
isempty(c::Claim) = c.width == 0 || c.height == 0

" calculate a sparse claim rectangle intersection "
function intersect(c1::Claim, c2::Claim)
    l = max(c1.left, c2.left)
    t = max(c1.top, c2.top)
    r = min(c1.left + c1.width, c2.left + c2.width)
    b = min(c1.top + c1.height, c2.top + c2.height)
    Claim("$(c1.id)∩$(c2.id)", l, t, max(r - l, 0), max(b - t, 0))
end

" calculate a dense claim rectangle union "
function union!(u::BitArray, c::Claim)
    for i in c.left + 1:c.left + c.width, j in c.top + 1:c.top + c.height
        u[i, j] = true
    end
    u
end

data = open("3.txt") do file
    readlines(file) .|> Claim
end


In [4]:
# calculate all intersections and reduce them into a non-overlapping union
intersections = (intersect(x, y)
                 for (i, x) in enumerate(data)
                 for y in data[i + 1:end])
            
area = reduce(union!, 
              intersections, 
              init=falses(1000, 1000)) |> sum


109143

In [5]:
# find the first claim with empty intersections for all others
disjoint = (x for x in data
            if all(isempty(intersect(x, y))
                   for y in data if x != y)) |> first


Claim("506", 261, 102, 10, 27)